In [3]:
import os
# Mount Google Drive
if not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')
else:
  print("Google Drive already mounted")

!ls /content/drive/MyDrive/

!cp /content/drive/MyDrive/NBA_Fantasy_Clustering.ipynb /content/NBA-Fantasy-Points-Prediction/notebooks/

# Navigate to the GitHub repository directory
%cd /content/NBA-Fantasy-Points-Prediction

# Add the notebook file to the repository
!git add notebooks/NBA_Fantasy_Clustering.ipynb

# Commit the changes
!git commit -m "Add clustering notebook for NBA Fantasy project"

# Push the changes to GitHub
!git push origin main


Cloning into 'NBA-Fantasy-Points-Prediction'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 113 (delta 54), reused 81 (delta 27), pack-reused 0 (from 0)
Receiving objects: 100% (113/113), 16.52 MiB | 19.71 MiB/s, done.
Resolving deltas: 100% (54/54), done.
cp: cannot stat '/content/NBA_Fantasy_Clustering.ipynb': No such file or directory
/content/NBA-Fantasy-Points-Prediction/NBA-Fantasy-Points-Prediction/NBA-Fantasy-Points-Prediction
fatal: pathspec 'NBA_Fantasy_Clustering.ipynb' did not match any files
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address
